In [1]:
# add this package to path
sys.path.insert(0, r'C:\Users\pasca\Dropbox\DataScienceAndProgramming\Projects\tox-block\tox-block')
from tox_block.config import config
from tox_block.data_processing import preprocessors as pp
from tox_block.data_processing import data_handling as dh
from tox_block.model import lstm_binary_label, lstm_multi_label
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import re


In [2]:
X_train, y_train = dh.load_data_binary_labels(config.TRAINING_DATA_FILE)
X_test, y_test = dh.load_data_binary_labels(config.TESTING_DATA_FILE)

In [3]:
#X_train_processed = pd.read_csv("../data/X_train.csv",skip_blank_lines=False)
#X_test_processed = pd.read_csv("../data/X_test.csv",skip_blank_lines=False)

In [4]:
ts = pp.TokenSequencer()
X_train_seq = ts.fit_transform(X_train)
X_test_seq = ts.transform(X_test)

In [5]:
em = dh.get_embedding_matrix(ts)

In [6]:
model = lstm_binary_label.get_model(embedding_matrix=em)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 50)           1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 100)          40400     
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dropout (Dropout)    

In [7]:
history = model.fit(X_train_seq, y_train.to_numpy(), 
                    batch_size=config.BATCH_SIZE, 
                    epochs=config.EPOCHS, 
                    validation_split=config.VALIDATION_SPLIT,
                    callbacks=lstm_binary_label.callbacks_list, 
                    #class_weight=config.CLASS_WEIGHT,
                    )

Train on 143613 samples, validate on 15958 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
143584/143613 [============================>.] - ETA: 0s - loss: 0.1235 - acc: 0.9556
Epoch 00001: val_loss improved from inf to 0.10245, saving model to C:\Users\pasca\Dropbox\DataScienceAndProgramming\Projects\tox-block\tox-block\tox_block\trained_models/checkpoint_BL.h5
143613/143613 [==============================] - 961s 7ms/sample - loss: 0.1234 - acc: 0.9556 - val_loss: 0.1024 - val_acc: 0.9639


In [8]:
model.evaluate(X_test_seq,y_test)

153164/153164 [==============================] - 144s 939us/sample - loss: 0.5965 - acc: 0.8077


[0.5964728326013228, 0.8076637]

In [9]:
y_pred = model.predict(X_test_seq)

In [10]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
print(f"ACC: {accuracy_score(y_test,np.round(y_pred))}")
print(f"F1: {f1_score(y_test,np.round(y_pred),average='macro')}")
print(f"ROC-AUC: {roc_auc_score(y_test,y_pred)}")




ACC: 0.807663680760492
F1: 0.5770046734568471
ROC-AUC: 0.8892040568683023


Let's do the same again with class weights

In [11]:
model = lstm_binary_label.get_model(embedding_matrix=em)
history = model.fit(X_train_seq, y_train.to_numpy(), 
                    batch_size=config.BATCH_SIZE, 
                    epochs=config.EPOCHS, 
                    validation_split=config.VALIDATION_SPLIT,
                    callbacks=lstm_binary_label.callbacks_list, 
                    class_weight=config.CLASS_WEIGHT,
                    )

Train on 143613 samples, validate on 15958 samples
143584/143613 [============================>.] - ETA: 0s - loss: 0.2364 - acc: 0.9127
Epoch 00001: val_loss did not improve from 0.10245
143613/143613 [==============================] - 900s 6ms/sample - loss: 0.2364 - acc: 0.9127 - val_loss: 0.1917 - val_acc: 0.9205


In [12]:
y_pred = model.predict(X_test_seq)

In [13]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
print(f"ACC: {accuracy_score(y_test,np.round(y_pred))}")
print(f"F1: {f1_score(y_test,np.round(y_pred),average='macro')}")
print(f"ROC-AUC: {roc_auc_score(y_test,y_pred)}")

ACC: 0.708645634744457
F1: 0.51693384471462
ROC-AUC: 0.8891845350004072
